In [2]:
%gui qt
from delta.assets import download_assets
from delta.utilities import cfg
from delta.data import seg_weights

from skimage.morphology import erosion, disk, binary_erosion

import pathlib
import json
import h5py

import numpy as np
from numpy.random import default_rng
import tifffile

import napari

In [3]:
def to_str(posixpath):
    return str(posixpath.resolve())  

In [15]:
#set paths
#root = pathlib.Path(pathlib.Path.home(), 'home', 'Caulobacter')

root = pathlib.Path(pathlib.Path.home(), 'Andreas', 'Delta2_Caulobacter')
data_dir = root / 'data'
model_dir = root / 'models'
training_set = data_dir / 'training'
validation_set = data_dir / 'validation'
savefile = model_dir / 'Caulobacter2D'

#path to training data
training_file = '/Users/simonvanvliet/Andreas/Training/training_data.hdf5'

#set fraction of training data to use for validation
validation_frac = 0.2

2022-06-21 20:41:46.643 python[21314:1216494] AdjustToIronwoodHotKeyChange - CG (hotmod:1) HotKey : hotKey enabled = 1, keyChar=0xfffbffff, virtKey=0x40000, flags=0x0 


In [5]:
#make folders
data_dir.mkdir(exist_ok=True)
model_dir.mkdir(exist_ok=True)

In [ ]:
#download delta assets   
download_assets(
    load_models=to_str(model_dir),
    load_sets=False,
    load_evals=False,
    config_level=to_str(root)
)

In [6]:
# modify config file
config_filename = root / 'config_2D.json'
with open(config_filename) as f:
    config = json.load(f)
    
#point to model location
config['model_file_seg'] = to_str(model_dir / 'unet_pads_seg.hdf5')    
config['model_file_track'] = to_str(model_dir / 'unet_pads_track.hdf5')    

#only needed for model training, in that case, point to location of training data
config['training_set_seg'] = to_str(data_dir / 'training')
config['training_set_track'] = ''

#point to raw data location
config['eval_movie'] = to_str(data_dir / 'evaluation')

#specify output formats
config['save_format'] = ["pickle", "movie"]    


new_config_filename = root / 'config_2D_caulobacter.json'
with open(new_config_filename, 'w') as f:
    json.dump(config, f, indent=2)   
           

In [6]:
#load data
with h5py.File(training_file, 'r') as f:
    labels = np.array(f['label'])
    images = np.array(f['images'])


Delta trains use a binary mask as GT. If in your label image cells tend to touch, they will become merged in the mask.
In this case it's better to first erode the cells by 1 pixel to make sure they don't touch.`

In [7]:
#process label masks
struc = disk(1)

eroded_labels = np.zeros_like(labels)

for idx, lab in enumerate(labels):
    #loop cells
    for c in np.unique(lab):
        if c>0:
            eroded_labels[idx, :, :] += c * binary_erosion(lab==c, disk(1))

In [34]:
#check new labels
viewer = napari.view_image(images, name="phase", colormap="gray")
viewer.add_labels(labels, name='orginal')   
viewer.add_labels(eroded_labels, name='eroded')   
napari.run()

In [8]:
# convert labels to binary mask
gt_mask = eroded_labels > 0 
gt_mask = gt_mask.astype(np.uint8)

In [14]:
#split data in training and validation set
rng = default_rng()

n_im = labels.shape[0]
n_val = int(np.ceil(n_im * validation_frac))  

valid_set = rng.choice(n_im, size=n_val, replace=False)
train_set = np.setdiff1d(np.arange(n_im), valid_set, assume_unique=True)

mask_train = gt_mask[train_set, :, :]
mask_valid = gt_mask[valid_set, :, :]

image_train = images[train_set, :, :]
image_valid = images[valid_set, :, :]


training_file = data_dir /  'training_data_delta.hdf5'

with h5py.File(training_file, 'w') as f:
    f.create_dataset('mask_train', data=mask_train)
    f.create_dataset('mask_valid', data=mask_valid)
    f.create_dataset('image_train', data=image_train)
    f.create_dataset('image_valid', data=image_valid)


In [12]:
mask_valid.shape

(23, 1024, 1024)

The seg_weights function of delta is super slow, so this code will take a while to run!

In [41]:
root = pathlib.Path(pathlib.Path.home(), 'Andreas', 'Delta2_Caulobacter')
data_dir = root / 'data'

training_file = data_dir /  'training_data_delta.hdf5'

with h5py.File(training_file, 'r') as f:
    mask_train = np.array(f['mask_train'])
    mask_valid = np.array(f['mask_valid'])
    image_train = np.array(f['image_train'])
    image_valid = np.array(f['image_valid'])    
    
training_set = data_dir / 'training'
validation_set = data_dir / 'validation'
    
training_set.mkdir(exist_ok=True)
validation_set.mkdir(exist_ok=True)

(training_set / 'img').mkdir(exist_ok=True)
(training_set / 'seg').mkdir(exist_ok=True)
(training_set / 'wei').mkdir(exist_ok=True)

(validation_set / 'img').mkdir(exist_ok=True)
(validation_set / 'seg').mkdir(exist_ok=True)
(validation_set / 'wei').mkdir(exist_ok=True)


#export training data to tiff    
for idx, t in enumerate(train_set):
    im_name = training_set / 'img' / ('img_%04i' % idx)
    lab_name = training_set / 'seg' / ('img_%04i' % idx)
    wei_name = training_set / 'wei' / ('img_%04i' % idx)
    
    tifffile.imwrite(im_name, np.squeeze(images[t,:,:]))
    tifffile.imwrite(lab_name, np.squeeze(gt_mask[t,:,:]))
    tifffile.imwrite(wei_name, seg_weights(np.squeeze(gt_mask[t,:,:])))    


for idx, v in enumerate(valid_set):
    im_name = validation_set / 'img' / ('img_%04i' % idx)
    lab_name = validation_set / 'seg' / ('img_%04i' % idx)
    wei_name = validation_set / 'wei' / ('img_%04i' % idx)
    
    tifffile.imwrite(im_name, np.squeeze(images[v,:,:]))
    tifffile.imwrite(lab_name, np.squeeze(gt_mask[t,:,:]))
    tifffile.imwrite(wei_name, seg_weights(np.squeeze(gt_mask[t,:,:])))
                

: 